In [61]:
import numpy as np
import math
import os
from IPython.display import clear_output

In [62]:
def getParams(stdMethod, sizeRange):
    paramDict = {}
    
    xNum = int(input("Number x nanoparticles: "))
    yNum = int(input("Number y nanoparticles: "))
    zNum = int(input("Number z nanoparticles: "))
    
    if sizeRange == True:
        dString = input("Desired Diameter range (in nm), use ' ' to separate: ")
        dDiameter = [float(i) for i in dString.split()]
        elDensity = float(input("Density of electrons (#/nm^3): "))
    else:
        dDiameter = float(input("Desired Diameter (in nm): "))
        elDensity = float(input("Density of electrons (#/nm^3): "))
    
    if stdMethod == "percent":
        dSTD = float(input("Percent Nanoparticle Size Variation (std): "))
    elif stdMethod == "fixed":
        dSTD = float(input("Nanoparticle Size Standard Deviation (in nm): "))
        
    lSTD = float(input("Nanoparticle Location Standard Deviation (in nm): "))
    vDensity = float(input("Percent Vacancies (in the QDxV(1-x) style): "))
    gbNumber = int(input("Number of Grain Boundaries?: "))
    tbNumber = int(input("Number of Twin Boundaries?: "))
    
    if gbNumber > 0:
        paramDict['voidCutoff'] = voidCut = float(input("What percentage of samples should have a void at the grain boundary?: "))
    else:
        paramDict['voidCutoff'] = voidCut = 0
    
    if gbNumber > 0 or tbNumber > 0:
        paramDict['angleRep'] = int(input("How many samples of the same angle?: "))
    else:
        paramDict['angleRep'] = 1
        
    npnumber = xNum*yNum*zNum
    
    paramDict['npnumber'] = npnumber
    paramDict['xNum'] = xNum
    paramDict['yNum'] = yNum
    paramDict['zNum'] = zNum
    paramDict['dDiameter'] = dDiameter
    paramDict['dSTD'] = dSTD
    paramDict['lSTD'] = lSTD
    paramDict['electronDensity'] = elDensity
    paramDict['vacancyDensity'] = vDensity
    paramDict['grainBoundaryNumber'] = gbNumber
    paramDict['twinBoundaryNumber'] = tbNumber
    paramDict['grainAngles'] = [np.random.uniform(-80*math.pi/180,80*math.pi/180)] #the default grain angle
    
    return paramDict

In [63]:
#the most general lattice, all other bravais lattices can be derived from this one
class triclinic_lattice:
    def __init__(self,latticeType,stdMethod,sizeRange,currDiameter,paramDict):
        self.npnumber = paramDict['npnumber']
        self.xNum = paramDict['xNum']
        self.yNum = paramDict['yNum']
        self.zNum = paramDict['zNum']
        self.dSTD = paramDict['dSTD']
        self.lSTD = paramDict['lSTD']
        self.elDensity = paramDict['electronDensity']
        self.vDensity = paramDict['vacancyDensity']
        self.gbNumber = paramDict['grainBoundaryNumber']
        self.tbNumber = paramDict['twinBoundaryNumber']
        self.gAngles = paramDict['grainAngles']
        self.tAngles = paramDict['twinAngles']
        self.voidCutoff = paramDict['voidCutoff']
        self.dDiameter = currDiameter
        self.lType = latticeType
        self.average_ll = 0.1 #expect ligand length total between np to be 1 nm, so per np = 0.5 nm
        self.bisect = 0 #will the grain bisect the sample in the transport direction? 0 = false, 1 = true
        
        #standard deviation of diameter
        if stdMethod == "percent":
            self.np_d_std = self.dDiameter*self.dSTD
        elif stdMethod == "fixed":
            self.np_d_std = self.dSTD
            
        #define upper and lower bounds of size disorder
        self.np_ub = self.dDiameter + 2*self.average_ll
        self.np_lb = self.dDiameter - 2*self.average_ll

        #define upper and lower bounds of position disorder
        self.npl_ub = self.average_ll/2
        self.npl_lb = -self.average_ll/2
        
        #define where the first nanoparticle will sit
        a = self.dDiameter/2.0 #+ average_ll
        self.baseArray = [a,a,a]
        
        self.np_array = []
        self.np_id_array = []
        
    #a method to define all the lattice angles
    def setLatticeAngles(self):
        self.alpha = 99*math.pi/180
        self.beta = 99*math.pi/180
        self.gamma = 99*math.pi/180
       
    #a method to set the basis vectors
    def setSpacingLattice(self):
        self.setLatticeAngles()
        
        #in the typical a,b,c representation, z will be in the b direction, x in a, and y in c
        self.sidelength = self.dDiameter + 2*self.average_ll
        self.a_1 = [0,0,self.sidelength]
        self.a_2 = [self.sidelength*math.sin(self.gamma),0,self.sidelength*math.cos(self.gamma)]
        
        x_3 = self.sidelength*(math.cos(self.beta)-math.cos(self.alpha)*math.cos(self.gamma))/math.sin(self.gamma)
        z_3 = self.sidelength*math.cos(self.alpha)
        y_3 = math.sqrt(self.sidelength**2-x_3**2-z_3**2)
        self.a_3 = [x_3,y_3,z_3]
        
    #defining the position where the lattice will be generated from
    def initiatePositions(self):
        self.curr_x = self.baseArray[0]
        self.curr_y = self.baseArray[1]
        self.curr_z = self.baseArray[2]

        self.cellx = self.baseArray[0]
        self.celly = self.baseArray[1]
        self.cellz = self.baseArray[2]
        
    #a method to add a nanoparticle to the nanoparticle solid. Called by generateLattice and add_zboundary_randomness
    def appendNP(self, npArray):
        #random size
        self.np_diameter = np.random.normal(self.dDiameter,self.np_d_std)
        while self.np_diameter < self.np_lb or self.np_diameter > self.np_ub:
            self.np_diameter = np.random.normal(self.dDiameter,self.np_d_std)
        
        #random location
        self.x_jit = np.random.normal(0,self.lSTD)
        self.y_jit = np.random.normal(0,self.lSTD)
        self.z_jit = np.random.normal(0,self.lSTD)
        self.total_jit = math.sqrt(self.x_jit**2 + self.y_jit**2 + self.z_jit**2)
        while self.total_jit < self.npl_lb or self.total_jit > self.npl_ub:
            self.x_jit = np.random.normal(0,self.lSTD)
            self.y_jit = np.random.normal(0,self.lSTD)
            self.z_jit = np.random.normal(0,self.lSTD)
            self.total_jit = math.sqrt(self.x_jit**2 + self.y_jit**2 + self.z_jit**2)
        
        #append nanoparticle
        npArray.append([self.curr_x + self.x_jit,self.curr_y+self.y_jit,self.curr_z+self.z_jit,self.np_diameter])
        return npArray
        
    #a method to create the nanoparticle solid
    #the id array is used for adding randomness to the left z-edge (needed for sorting the lattice)
    #cell_edge values are used for calculating sample volume
    def generateLattice(self):
        npArray = []
        idArray = []
        base_id = -1
        id = 0
        
        base_x = self.baseArray[0]
        base_z = self.baseArray[2]
        base_z_y = self.baseArray[2]
        
        for y in range(1,self.yNum+1):
            for x in range(1,self.xNum+1):
                base_id += 1
                id = base_id
                for z in range(1,self.zNum+1):
                    npArray = self.appendNP(npArray)
                    idArray.append(id)
                    id+=self.xNum
                    if y == 1 and x == 1 and z == self.zNum:
                        self.cellz = self.curr_z - self.cellz + self.dDiameter + self.z_jit
                        #due to triclinic nature, for volume of cell we want the length of the sides
                        self.cellz_edge = (self.zNum-1)*(self.sidelength) + self.dDiameter + self.z_jit
                    
                    self.curr_x += self.a_1[0]
                    self.curr_y += self.a_1[1]
                    self.curr_z += self.a_1[2]
                
                if y == 1 and x == self.xNum:
                    self.cellx = self.curr_x - self.cellx + self.dDiameter + self.x_jit
                    
                    #length of side in the x direction
                    self.cellx_edge = (self.xNum-1)*(self.sidelength) + self.dDiameter + self.x_jit
                   
                base_z += self.a_2[2]
                
                self.curr_z = base_z
                self.curr_x += self.a_2[0]
                self.curr_y += self.a_2[1]
                
            base_x += self.a_3[0]
            base_z_y += self.a_3[2]
            base_z = base_z_y
            
            self.curr_x = base_x
            self.curr_z = base_z
            
            self.curr_y += self.a_3[1]
            
            base_id += (self.xNum*self.zNum-base_id)
            
        self.celly = self.curr_y - self.a_3[1] - self.celly + self.dDiameter + self.y_jit
        
        #length of side in the y direction
        self.celly_edge = (self.yNum-1)*(self.sidelength) + self.dDiameter + self.y_jit
        
        return npArray,idArray
        
    #a method to calculate the volume of the nanoparticle solid
    def calculateVolume(self):
        return self.cellx_edge*self.celly_edge*self.cellz_edge*math.sqrt(1-math.cos(self.alpha)**2 - math.cos(self.beta)**2-math.cos(self.gamma)**2 + 2*math.cos(self.alpha)*math.cos(self.beta)*math.cos(self.gamma))
    
    #return which y-layer a nanoparticle belongs to, critical if including location disorder
    def calculateYLayer(self, y):
        return int((y - self.baseArray[1])/self.a_3[1] + 0.5) #note: need to add 0.5 to round (0.5,1) up to 1
    
    #a method to remove individual nanoparticles at random
    def createVacancies(self,np_array,id_array):
        indexArray = [] #to pop nanoparticle ids out when relevant
        numberVacancies = int(self.vDensity*len(np_array))
        for x in range(0,numberVacancies):
            randIndex = np.random.randint(0,len(np_array))
            #make sure that the nanoparticle being removed is not at either end in the z direction (will affect transport code)
            zValue = np_array[randIndex][2]
            while zValue < self.dDiameter or zValue > (self.cellz - self.dDiameter):
                randIndex = np.random.randint(0,len(np_array))
                zValue = np_array[randIndex][2]
            np_array.pop(randIndex)
            id_array.pop(randIndex)
        return np_array, id_array
    
    #a method to define where the grains will seed from in the lattice
    def defineGrains(self):
        self.grainList = []
        
        #as a crude way of spacing out grains, make sure they occur at regular intervals in z
        for x in range(0,self.gbNumber):
            randIndex = np.random.randint(0,len(self.np_array))
            yValue = self.np_array[randIndex][1]
            zValue = self.np_array[randIndex][2]
            grainSpace = self.cellz/(self.gbNumber+1)
            while yValue > self.dDiameter or zValue < (x+1)*grainSpace or zValue > (x+3/2)*(grainSpace):
                randIndex = np.random.randint(0,len(self.np_array))
                zValue = self.np_array[randIndex][2]
                yValue = self.np_array[randIndex][1]
                
            self.grainList.append(randIndex)
            
    #a method to define where the twin will seed from in the lattice
    def defineTwin(self):
        self.twinList = []
        
        for x in range(0,self.tbNumber):
            #want to target the top right corner of the top layer
            self.twinList.append(int(len(self.np_array)-1))
        
    #a method to create a void between two grains
    def voidSeparation(self,theta):
        voidBoolFloat = np.random.uniform()
        
        if voidBoolFloat > (1-self.voidCutoff):
            sepMean = 2.5*self.dDiameter
            sepSTD = 0.5*self.dDiameter
            gSeparation = np.abs(np.random.normal(sepMean,sepSTD))
        else:
            gSeparation = 0
            
        return gSeparation
     
    #delete points along and over grain line
    #will remove points below or above line in the x-direction depending on rotation angle theta
    #will remove nanoparticles that intersect in the two arrays, from the grainArray (not rotated array)
    def removeNanoparticlesOnGrainLine(self, theta,grainArray,rotated_np_array,lineValues):
        newArray = []
        z1 = lineValues[0]
        x1 = lineValues[1]
        m1 = lineValues[2]
        line = np.asarray([x1,z1])
        lineVector = np.asarray([1, m1])
        
        #allow for void space between grains
        tSeparation = self.voidSeparation(theta)
        
        for i in grainArray:
            #get all the coordinates as variables x,y,z
            x = i[0]
            y = i[1]
            z = i[2]
            yN = self.calculateYLayer(y) #y layer number, starting with 0
            remove = False
            
            if theta > 0 and x > ((z-(z1+yN*self.a_3[2]))/m1 + (x1+yN*self.a_3[0])):
                remove = True
            elif theta < 0 and x  < ((z-(z1+yN*self.a_3[2]))/m1 + (x1+yN*self.a_3[0])):
                remove = True   
                
            #to create a void, calculate the distance from nanoparticle to grain boundary
            point = np.asarray([x,z])
            projPointOntoLine = line + lineVector*np.dot(point - line,lineVector)/np.dot(lineVector,lineVector)
            distV = projPointOntoLine - point
            dist = math.sqrt(np.dot(distV,distV))    
            if dist < tSeparation:
                remove = True

            #remove on line in grainArray, by checking if the nanoparticles overlap directly between two arrays
            if not remove: 
                for j in rotated_np_array:
                    if yN == self.calculateYLayer(j[1]):
                        distance = math.sqrt((j[0]-x)**2 +(j[2]-z)**2)
                        if distance < self.dDiameter:
                            remove = True
                            break
            
            if not remove:
                newArray.append(i)

        return newArray
    
    #delete points along and over twin line
    def removeNanoparticlesOnTwinLine(self, theta,twinArray,lineValues):
        newArray = []
        
        z1 = lineValues[0]
        x1 = lineValues[1]
        m1 = lineValues[2]
        line = np.asarray([x1,z1])
        lineVector = np.asarray([1, m1])
        
        for i in twinArray:
            #get all the coordinates as variables x,y,z
            x = i[0]
            y = i[1]
            z = i[2]
            yN = self.calculateYLayer(y) - (self.yNum - 1) #y layer number, ending at 0 (so ..., -2, -1, 0)
            remove = False
            
            #calculate distance to the twin line. Only will remove nanoparticles if the distance 
            #of the nanoparticle to the twin line is greater than a half diameter
            point = np.asarray([x,z])
            projPointOntoLine = line + lineVector*np.dot(point - line,lineVector)/np.dot(lineVector,lineVector)
            distV = projPointOntoLine - point
            dist = math.sqrt(np.dot(distV,distV))
            
            if theta > 0 and x > ((z-(z1+yN*self.a_3[2]))/m1 + (x1+yN*self.a_3[0])) and dist > self.dDiameter/2:
                remove = True
            elif theta < 0 and x < ((z-(z1+yN*self.a_3[2]))/m1 + (x1+yN*self.a_3[0]))  and dist > self.dDiameter/2:
                remove = True    
            
            if not remove:
                newArray.append(i)

        return newArray
    
    #remove points outside crystal bounds, for a specified array
    def removeNanoparticlesOutside(self,grainArray):
        newArray = []
        
        #right z-edge line
        x2 = self.baseArray[0]
        z2 = self.cellz - self.baseArray[2]
        xEdgeAngle = self.gamma
                
        if xEdgeAngle != math.pi/2:
            m2 = 1/math.tan(xEdgeAngle)
        else:
            m2 = 0
            
        #left z-edge line
        x3 = self.baseArray[0]
        z3 = self.baseArray[2]
        m3 = m2
        
        for i in grainArray:
            remove = False
            x = i[0]
            y = i[1]
            z = i[2]
            yN = self.calculateYLayer(y)

            #Standard definition of crystal bounds  as linear lines. yN gives the y layer number
            #so that the layers can be shifted. Using the location disorder upper and lower bounds to allow
            #for jitter if applicable
            leftz = m3*(x-(x3+yN*self.a_3[0])) + (z3+yN*self.a_3[2]) + self.npl_lb - .001
            rightz = m2*(x-(x2+yN*self.a_3[0])) + (z2+yN*self.a_3[2]) + self.npl_ub + .001
            topx = self.cellx - self.baseArray[0] + yN*self.a_3[0] + self.npl_ub + .001
            bottomx = self.baseArray[0] + yN*self.a_3[0] +self.npl_lb - .001

            if not(z < leftz or z > rightz or x > topx or x < bottomx) :
                newArray.append(i)
                
        return newArray
    
            
    #a method to add unevenness to the left z-boundary of an array
    #used in the formation of grain boundaries
    def add_zBoundary_randomness(self, npList, idList,size, phi):
        
        #add randomness without phi included, so that magnitude of rotation is correct (seems to be off)
        sorted_npArray = [x for _,x in sorted(zip(idList,npList))]
        npList[:] = sorted_npArray
        
        baseSize = int(len(npList)/self.yNum)
        workingList = npList[0:size] + npList[baseSize:(baseSize+size)]
        
        for j in workingList:
            
            r = np.random.random_sample()
            if r <= 1.5/3:
                #add nanoparticle to the left
                np_diameter = np.random.normal(self.dDiameter,self.np_d_std)
                while np_diameter < self.np_lb or np_diameter > self.np_ub:
                    np_diameter = np.random.normal(self.dDiameter,self.np_d_std)

                #random location in 3D space
                x_jit = np.random.normal(0,self.lSTD)
                y_jit = np.random.normal(0,self.lSTD)
                z_jit = np.random.normal(0,self.lSTD)
                total_jit = math.sqrt(self.x_jit**2 + self.y_jit**2 + self.z_jit**2)
                while total_jit < self.npl_lb or total_jit > self.npl_ub:
                    x_jit = np.random.normal(0,self.lSTD)
                    y_jit = np.random.normal(0,self.lSTD)
                    z_jit = np.random.normal(0,self.lSTD)
                    total_jit = math.sqrt(self.x_jit**2 + self.y_jit**2 + self.z_jit**2)

                b_x = -(self.a_1[0]*math.cos(phi)+self.a_1[2]*math.sin(phi))
                b_z = -(-self.a_1[0]*math.sin(phi)+self.a_1[2]*math.cos(phi))

                np_x = j[0] + x_jit + b_x
                np_y = j[1] + y_jit
                np_z = j[2] + z_jit + b_z
                #print([np_x,np_y,np_z])
                
                #append nanoparticle
                npList.append([np_x,np_y,np_z,np_diameter])
                    
        return npList
            
    #the method to create grains inside the nanoparticle samples. 
    def createGrains(self):
        #initialize grain seed points
        grainArray = self.np_array
        self.defineGrains()
        
        #iterate through all grain seed points. j is the index of the nanoparticle inside np_array
        for index, j in enumerate(self.grainList):
            #the angle is defined outside the class, for repeatability
            theta = self.gAngles[index]
            
            #grain line
            #line equation: z-z1 = m1*(x-x1)
            i = self.np_array[j]
            x1 = i[0]
            y1 = i[1]
            z1 = i[2]
            m1 = -math.tan(theta)#(z/x)
            
            #right x-edge line
            x2 = self.baseArray[0]
            z2 = self.cellz - self.baseArray[2]
            xEdgeAngle = self.gamma
                
            if xEdgeAngle != math.pi/2:
                m2 = 1/math.tan(xEdgeAngle)
            else:
                m2 = 0
            
            #left x-edge line
            x3 = self.baseArray[0]
            z3 = self.baseArray[2]
            m3 = m2
            
            
            #find lattice edge intercepts
            have_top = False
            have_bottom = False
            
            #checking if intersection of grain line with right x-edge line exists
            x_right = (z1-z2-m1*x1+m2*x2)/(m2-m1)
            if x_right >= self.baseArray[0] and x_right <= x1:
                #bottom intersects right x-edge
                z_b = m1*(x_right-x1)+z1
                x_b = x_right
                have_bottom = True
                
            elif x_right >= x1 and x_right <= self.cellx + self.baseArray[0]:
                #top intersects right x-edge
                z_t = m1*(x_right-x1)+z1
                x_t = x_right
                have_top = True
             
            #checking if intersection with left x-edge line exists
            x_left = (z1-z3-m1*x1+m3*x3)/(m3-m1)
            if x_left >= x1 and x_left <= self.cellx+self.baseArray[0]:
                #top intersects left x-edge
                z_t = m1*(x_left-x1) + z1
                x_t = x_left
                have_top = True
                
            elif x_left <= x1 and x_left >= self.baseArray[0]:
                #bottom intersects left x-edge
                z_b = m1*(x_left-x1)+z1
                x_b = x_left
                have_bottom=True
            
            #check if intersection instead with bottom z-edge
            if not have_bottom:
                x_b = self.baseArray[0]
                z_b = m1*(x_b-x1)+z1
                
            #check if intersection instead with top z-edge
            if not have_top:
                x_t = self.cellx-self.baseArray[0]
                z_t = m1*(x_t-x1)+z1
                
            #now start to define how much big the rotated lattice needs to be to cover minimally the entire region
            line_length = math.sqrt((x_t-x_b)**2 + (z_t-z_b)**2)
            bottom_overhang = 0
            
            #defining the rotation angle for the lattice. It will be a counterclockwise rotation.
            phi = theta + math.pi/2 - self.gamma
            
            #Now we will calculate how much overhang is needed, and adjust bottom_overhang accordingly
            #top x-edge
            if not have_top and phi < 0:
                #need to have more overhang. Relevant lattice angle is math.pi - gamma
                z_distance = self.cellz_edge + self.baseArray[2] - z_t
                crit_angle = 90*math.pi/180 + theta
                x_angle = math.pi - crit_angle - (math.pi - self.gamma) 
                if x_angle > 0:
                    x_overhang = math.sin(x_angle)*z_distance/math.sin(math.pi - self.gamma)
                else:
                    x_overhang = 0
                line_length += x_overhang
            
            #bottom x-edge
            if not have_bottom and phi > 0:
                #need to have more overhang. Relevant lattice angle is gamma
                z_distance = self.cellz_edge + self.baseArray[2] - z_b
                crit_angle = math.pi/2 - theta
                x_angle = math.pi - crit_angle - self.gamma
                if x_angle > 0:
                    x_overhang = math.sin(x_angle)*z_distance/math.sin(self.gamma)
                else:
                    x_overhang = 0
                line_length += x_overhang
                bottom_overhang += x_overhang
               
            #left z-edge
            if have_top and phi > 0:
                #need to have more overhang
                #define the top left z location of the lattice
                topleftz = self.baseArray[2] + (self.xNum-1)*self.a_2[2]
                
                x_distance = math.sqrt((self.cellx_edge - self.baseArray[0] - x_t)**2 + (topleftz - z_t)**2)
                crit_angle = phi
                x_angle = self.gamma - crit_angle
                if x_angle > 0:
                    x_overhang = math.sin(x_angle)*x_distance/math.sin(math.pi-self.gamma)
                else:
                    x_overhang = 0
                line_length += x_overhang
            
            if have_bottom and phi < 0:
                #need to have more overhang
                x_distance = math.sqrt((x_b - self.baseArray[0])**2 + (z_b - self.baseArray[2])**2)
                crit_angle = theta + self.gamma - math.pi/2
                x_angle = math.pi - crit_angle - self.gamma
                if x_angle > 0:
                    x_overhang = math.sin(x_angle)*x_distance/math.sin(self.gamma)
                else:
                    x_overhang = 0
                line_length += x_overhang
                bottom_overhang += x_overhang
                
            #right z-edge
            if have_top and phi < 0:
                #need more overhang
                x_distance = math.sqrt((x_t - self.baseArray[0])**2 + (z_t - self.cellz_edge)**2)
                crit_angle = math.pi - (theta + self.gamma - math.pi/2)
                x_angle = self.gamma - crit_angle
                if x_angle > 0:
                    x_overhang = math.sin(x_angle)*x_distance/math.sin(math.pi - self.gamma)
                else:
                    x_overhang = 0
                line_length += x_overhang

            #create rotated array of appropriate size
            #make sure to rotate by the triclinic lattice angle as well
            
            #condition for bisection of the transport direction
            if not have_top and not have_bottom:
                self.bisect = 1
                
            if line_length <= self.cellx_edge:
                #add randomness to the left z-edge of the nanoparticle array (to be rotated)
                xSize = self.xNum
                npArrayCopy = self.add_zBoundary_randomness(self.np_array, self.np_id_array,xSize, 0)
                
            else:
                #store the original cell values
                cellArray = [self.cellx,self.celly,self.cellz]
                
                #create new lattice of appropriate size
                old_xNum = self.xNum
                self.xNum = int(math.ceil((line_length + bottom_overhang - self.dDiameter)/self.sidelength + 2))
                self.initiatePositions()
                new_npArray, new_idArray = self.generateLattice()
                new_npArray[:], new_idArray[:] = self.createVacancies(new_npArray, new_idArray)
                
                #add randomness to the left z-edge of the nanoparticle array (to be rotated)
                xSize = self.xNum
                npArrayCopy = self.add_zBoundary_randomness(new_npArray, new_idArray,xSize, 0)
                
                #recover the original cell values and xNum
                [self.cellx,self.celly,self.cellz] = cellArray
                self.xNum = old_xNum
                
            rotate_length = math.sqrt((x1-x_b)**2 + (z1-z_b)**2)
            rotate_length += bottom_overhang
            
            if self.gamma < math.pi/2:
                x_rotate = self.baseArray[0] + rotate_length*math.sin(self.gamma)
                z_rotate = self.baseArray[2] + rotate_length*math.cos(self.gamma)
            else:
                x_rotate = self.baseArray[0] + rotate_length*math.cos(self.gamma-math.pi/2)
                z_rotate = self.baseArray[2] - rotate_length*math.sin(self.gamma-math.pi/2)
                     
            rotated_np_array = [ [i[0] - x_rotate, i[1], i[2] - z_rotate,i[3]] for i in npArrayCopy]
            
            #perform rotation in place
            rotated_np_array[:] = [[i[0]*math.cos(phi)+i[2]*math.sin(phi),i[1],-i[0]*math.sin(phi)+i[2]*math.cos(phi),i[3]] for i in rotated_np_array]
            rotated_np_array[:] = [ [i[0] + x_rotate, i[1], i[2] + z_rotate,i[3]] for i in rotated_np_array] 
                
            #move rotated array to correct position
            x_add = x1 - x_rotate
            z_add = (z1 - z_rotate)
            rotated_np_array[:] = [[i[0] + x_add, i[1], i[2] + z_add,i[3]] for i in rotated_np_array]
            
            #delete points along and over line, and add rotated array
            lineValues = [z1,x1,m1]
            grainArray[:] = self.removeNanoparticlesOnGrainLine(theta,grainArray,rotated_np_array,lineValues)
            grainArray[:] = grainArray + rotated_np_array
            
            #remove points outside crystal bounds
            leftzLine = [z3,x3,m3]
            rightzLine = [z2,x2,m2]      
            grainArray[:] = self.removeNanoparticlesOutside(grainArray)
         
        #update the final crystal
        self.np_array = grainArray
        
    #the method to create twins inside the nanoparticle samples. 
    def createTwin(self):
        #initialize twin boundary seed points, recyling the grain boundary method
        self.defineTwin()
        for index, j in enumerate(self.twinList):
            theta = self.tAngles[index]
            #theta = -40.5*math.pi/180.0 #45 degrees
            i = self.np_array[j]

            #twin line
            x0 = i[0]
            y0 = i[1]
            z0 = i[2]
            m0 = -math.tan(theta)#(z/x)

            #right z-edge line
            x1 = self.baseArray[0]
            z1 = self.cellz - self.baseArray[2]
            xEdgeAngle = self.gamma

            if xEdgeAngle != math.pi/2:
                m1 = 1/math.tan(xEdgeAngle)
            else:
                m1 = 0

            #left z-edge line
            x2 = self.baseArray[0]
            z2 = self.baseArray[2]
            m2 = m1

            #create a lattice twice as large
            old_xNum = self.xNum
            old_zNum = self.zNum
            self.xNum = 2*old_xNum
            self.zNum = 2*old_zNum

            #store the original cell values
            cellArray = [self.cellx,self.celly,self.cellz]
            
            #initiate lattice placement
            self.initiatePositions()

            #generate lattices (with vacancies)
            twinArray, new_idArray = self.generateLattice()
            twinArray[:], new_idArray[:] = self.createVacancies(twinArray, new_idArray)

            #recover the original cell values, xNum and zNum
            [self.cellx,self.celly,self.cellz] = cellArray
            self.xNum = old_xNum
            self.zNum = old_zNum

            #remove nanoparticles on and past the twin line
            lineValues = [z0,x0,m0]
            twinArray[:] = self.removeNanoparticlesOnTwinLine(theta,twinArray,lineValues)
            
            #define the line vector
            lineV = np.asarray([1, m0])
            
            #perform reflection via projection, e.g. Ref(v) = 2*dot(v,l)/dot(l,l)*l - v
            mirroredArray = []
            for i in twinArray:
                #make line seed the origin
                l = np.asarray([x0,z0])
                point = np.asarray([i[0],i[2]])
                
                projLine = l + lineV*np.dot(point - l,lineV)/np.dot(lineV,lineV)
                refV = (2*projLine - point).tolist()
                
                #make sure nanoparticles are different diameters, to preserve statistics
                np_diam = np.random.normal(self.dDiameter,self.np_d_std)
                while np_diam < self.np_lb or np_diam > self.np_ub:
                    np_diam = np.random.normal(self.dDiameter,self.np_d_std)
                nano = [refV[0],i[1],refV[1],np_diam]
                mirroredArray.append(nano)

            #shift the two arrays by 10 nanoparticles along the diagonal direction, so that the seed is 
            #approximately in the center of the sample
            x_add = -10*(self.a_2[0])
            z_add = -10*(self.a_1[2] + self.a_2[2])
            twinArray[:] = [[i[0] + x_add, i[1], i[2] + z_add,i[3]] for i in twinArray]
            mirroredArray[:] = [[i[0] + x_add, i[1], i[2] + z_add,i[3]] for i in mirroredArray]
            
            #remove any nanoparticles outside of the crystal bounds for both of the two arrays
            leftzLine = [z2,x2,m2]
            rightzLine = [z1,x1,m1]      
            twinArray[:] = self.removeNanoparticlesOutside(twinArray)
            mirroredArray[:] = self.removeNanoparticlesOutside(mirroredArray)
            
            #now join the two arrays and delete any overlapping nanoparticles
            totalArray = twinArray
            for i in mirroredArray:
                remove = False
                x = i[0]
                y = i[1]
                z = i[2]

                #remove on line, by checking if the nanoparticles overlap directly
                #will only compare nanoparticles inside the same layers
                for j in twinArray:
                    if self.calculateYLayer(y) == self.calculateYLayer(j[1]):
                        distance = math.sqrt((j[0]-x)**2 +(j[2]-z)**2)
                        if distance < self.dDiameter:
                            remove = True
                            break
                if not remove:
                    totalArray.append(i)

            #update the final crystal
            self.np_array = totalArray
                                    
    def returnLattice(self):
        self.setSpacingLattice()
        self.initiatePositions()
        self.np_array, self.np_id_array = self.generateLattice()
        self.np_array[:], self.np_id_array[:] = self.createVacancies(self.np_array, self.np_id_array)
        self.cellV = self.calculateVolume()
        self.cellsize_array = [self.cellx, self.celly, self.cellz]
        self.createGrains()
        self.createTwin()
        
        #create the flipped array
        if self.lType=="cubic" or self.lType == "cpc":
            self.np_array_flipped = [[i[0], i[1], self.cellsize_array[2]-i[2], i[3]] for i in self.np_array] #flipped in the transport direction
        elif self.lType=="tric":
            #right x-edge line
            x0 = self.baseArray[0]
            z0 = self.cellz #- self.baseArray[2]
            m = 1/math.tan(self.gamma)

            #z_right = z0 + m*(i[0]-x0), add an extra m*(i[0]-x0) so that it is still triclinic
            self.np_array_flipped = [[i[0], i[1], z0 + 2*m*(i[0]-x0)-i[2], i[3]] for i in self.np_array] #flipped in the transport direction
        
        #print("Number of nanoparticles is: %i"%len(self.np_array))
        return self.np_array, self.np_array_flipped, self.cellsize_array,self.cellV, self.bisect
        
        
class cubic_lattice(triclinic_lattice):
    #def _init_(self,stdMethod,sizeRange,currDiameter,paramDict):
    #    super()._init_(stdMethod,sizeRange,currDiameter,paramDict)
    def setLatticeAngles(self):
        #define lattice angles
        self.alpha = 90*math.pi/180
        self.beta = 90*math.pi/180
        self.gamma = 90*math.pi/180
    
class cpc_lattice(triclinic_lattice):
    #def _init_(self,stdMethod,sizeRange,currDiameter,paramDict):
    #    super()._init_(stdMethod,sizeRange,currDiameter,paramDict)
    def setLatticeAngles(self):
        #define lattice angles
        self.alpha = 90*math.pi/180
        self.beta = 90*math.pi/180
        self.gamma = 90*math.pi/180
        
    def setLatticeSpacing(self):
        self.x_spacing = self.dDiameter + 2*self.average_ll
        self.y_spacing = layer_spacing = (dDiameter + 2*average_ll)*math.sqrt(3)/2
        self.z_spacing = self.dDiameter + 2*self.average_ll
        
    def generateLattice(self):
        base_id = -1
        id = 0
        for y in range(1,self.yNum+1):
            for x in range(1,self.xNum+1):
                base_id += 1
                id = base_id
                for z in range(1,self.zNum+1):
                    npArray = self.appendNP(npArray)
                    idArray.append(id)
                    id+=self.xNum
                    if y == 2 and x == 1 and z == self.zNum:
                        cellz = curr_z - cellz + dDiameter + self.z_jit
                        self.cellz_edge = (self.zNum-1)*(self.sidelength) + self.dDiameter + self.z_jit
                    curr_z += self.z_spacing
                    
                self.curr_z = self.baseArray[2]
                if y == 2 and x == self.xNum:
                    self.cellx = self.curr_x - self.cellx + self.dDiameter + self.x_jit
                    self.cellx_edge = (self.xNum-1)*(self.sidelength) + self.dDiameter + self.x_jit
                curr_x += bottom_spacing
            if y%2 != 0:
                self.curr_x = self.baseArray[0] + self.x_spacing/2.0
                self.curr_z = self.baseArray[2] + self.z_spacing/2.0
            else:
                self.curr_x = self.baseArray[0]
                self.curr_z = self.baseArray[2]
                
            self.curr_y += self.y_spacing
            
        self.celly = self.curr_y - self.y_spacing - self.celly + self.dDiameter + self.y_jit  
        self.celly_edge = (self.yNum-1)*(self.sidelength) + -self.celly + self.dDiameter + self.y_jit
        return npArray,idArray

In [64]:
def createSample(method,stdMethod,sizeRange, currDiameter, paramDict):
    if method == "cubic":
        sample = cubic_lattice(method,stdMethod,sizeRange, currDiameter, paramDict)
    elif method == "cpc":
        sample = cpc_lattice(method,stdMethod,sizeRange, currDiameter, paramDict)
    elif method == "tric":
        sample = triclinic_lattice(method,stdMethod,sizeRange, currDiameter, paramDict)
    
    return sample.returnLattice()



In [65]:
def grainAngleRepetition(method,stdMethod,sizeRange, currDiameter, paramDict):
    gbNumber = paramDict['grainBoundaryNumber']
    tbNumber = paramDict['twinBoundaryNumber']
    angleRep = paramDict['angleRep']
    
    #define grain angle as a random angle between -80 and 80 degrees. This will be a rotation counterclockwise
    grainAngles = []
    i = 0
    if gbNumber > 0:
        while i < gbNumber:
            grainAngles.append(np.random.uniform(-80*math.pi/180,80*math.pi/180))
            i+=1  
    paramDict['grainAngles'] = grainAngles
    
    twinAngles = []
    j = 0
    if tbNumber > 0:
        while j < tbNumber:
            twinAngles.append(np.random.uniform(-80*math.pi/180,80*math.pi/180))
            j+=1
    paramDict['twinAngles'] = twinAngles
     
    sampleArray = []
    bisectArray = []
    total_cellV = 0.0
    a = 0
    while a < angleRep:
        np_array, np_array_flipped, cellsize_array, cellV, bisect= createSample(method,stdMethod,sizeRange, currDiameter, paramDict)
        sampleArray.append([np_array, cellsize_array])
        sampleArray.append([np_array_flipped, cellsize_array])
        total_cellV += cellV
        bisectArray.append(bisect)
        
        a+=1
        
    return sampleArray, total_cellV, grainAngles, bisectArray
    

In [66]:
def generateSamples(method,stdMethod,sizeRange, currDiameter, paramDict, length):
    #generate samples, with inverted pairs
    sampleArray = []
    angleArray = []
    bisectArray = []
    s = 0
    total_cellV = 0
    while s < (numberSamples/2):
        print("\r" + "Creating sample pair %03d and %03d"%(2*s,2*s+1),end="")
        sArray, cellV, grainAngles,bisect = grainAngleRepetition(method,stdMethod,sizeRange, currDiameter, paramDict)
        sampleArray.extend(sArray)
        angleArray.append(grainAngles)
        bisectArray.append(bisect)
        total_cellV += cellV
        s += 1
        
    print("")

    #print out electron numbers for each nanoparticle, will be slightly different for each sample, so average over volume
    average_cellV = 2*total_cellV/(length)
    elNumber = elDensity*average_cellV
    print("NP Diameter: %f,  Number of electrons: %d" %(currDiameter, elNumber))
    
    return sampleArray, angleArray, bisectArray

def outputNanoparticleSamples(filepath, length, sampleArray):
    i = 0
    while i < (length):
        filename = "nanoparticles" + str(i) + ".inp"
        fullpath = os.path.join(filePath, filename)
        
        np_array = sampleArray[i][0]
        cellsize_array = sampleArray[i][1]

        with open(fullpath,'w+') as f:
            f.write("BANDS\n")
            f.write("8 8\n") #degeneracy , always 8 8 for PbSe
            f.write("NANOPARTICLES\n")
            f.write("cell(1), %f\n" %cellsize_array[0])#X  center-center distance leftmost-rightmost + desired diameter
            f.write("cell(2), %f\n" %cellsize_array[1])#Y  Defines height (layers)
            f.write("cell(3), %f\n" %cellsize_array[2])#Z  Transport direction

            for j in range(0,len(np_array)):
                x = str(np_array[j][0])
                y = str(np_array[j][1])
                z = str(np_array[j][2])
                d = str(np_array[j][3])
                f.write(x + ", " + y + ", " + z + ", " + d + "\n")
                
        i += 1
        
def outputSampleAngles(filepath, angleArray, bisectArray):
    filename = "SampleAngles.txt"
    fullpath = os.path.join(filePath, filename)
    
    #merge angleArray and bisectArray
    for n,i in enumerate(angleArray):
        i.append(bisectArray[n])

    with open(fullpath,'w+') as f:
        for j in range(1,len(angleArray)+1):
            #angleArray[j-1].append(bisectArray[j-1])
            string_of_angles = str(angleArray[j-1][0])
            for i in range(1,len(angleArray[j-1])):
                string_of_angles += " "
                string_of_angles += str(angleArray[j-1][i])    
            f.write(str(j) + " " + string_of_angles + "\n")

In [67]:
stdMethod = "fixed"
#"percent": percent of mean
#"fixed": user set value

sizeRange = False
#True: will generate nanoparticles samples in a size range (for diameter)
#False: will generate nanoparticles at only one diameter

method = "tric"
#cubic = simple cubic
#cpc = close packed cubic (body centered)
#tric = triclinic, angles are defined inside the triclinic sample generation code
    
#get user inputs regarding parameters, and read into dictionary
paramDict = getParams(stdMethod, sizeRange)
numberSamples = int(input("How many samples (must be even number)?: "))  #number of samples (not including angle rep)

#use user built dictionary to define quantities
angleRep = paramDict['angleRep'] #number of times grain angle will be repeated between samples
length = numberSamples*angleRep #total samples (including angle rep)
npnumber = paramDict['npnumber'] #number of nanoparticles
dDiameter = paramDict['dDiameter'] #nanoparticle diameter
elDensity = paramDict['electronDensity'] #electron density

#choose to create samples for either a single diameter, or a range of diameters
if sizeRange: 
    currDiameter = dDiameter[0]
    while currDiameter <= dDiameter[1]:
        
        sampleArray, angleArray, bisectArray = generateSamples(method,stdMethod,sizeRange, currDiameter, paramDict, length)
        
        #assign file path
        filePath = 'c:/Users/Davis/Research/data/nanoparticles/' + str(npnumber) + '_' + str(currDiameter) + 'nm/'
        if not os.path.exists(filePath):
            os.makedirs(filePath)
        
        #################
        # output angles #
        #################
        outputSampleAngles(filePath, angleArray,bisectArray)
        
        ##################
        # output samples #
        ##################
        outputNanoparticleSamples(filePath, length, sampleArray)
        
        #np diameter increment
        currDiameter += 0.5
        
else:
    filePath = 'c:/Users/Davis/Research/data/nanoparticles/' + str(npnumber) + '_' + str(dDiameter) + 'nm/'
    if not os.path.exists(filePath):
        os.makedirs(filePath)
        
    sampleArray, angleArray, bisectArray = generateSamples(method,stdMethod,sizeRange, dDiameter, paramDict, length)
    
    #################
    # output angles #
    #################
    outputSampleAngles(filePath, angleArray,bisectArray)

    ##################
    # output samples #
    ##################
    
    outputNanoparticleSamples(filePath, length, sampleArray)

Number x nanoparticles: 20
Number y nanoparticles: 2
Number z nanoparticles: 20
Desired Diameter (in nm): 6.6
Density of electrons (#/nm^3): .002
Nanoparticle Size Standard Deviation (in nm): .3
Nanoparticle Location Standard Deviation (in nm): 0
Percent Vacancies (in the QDxV(1-x) style): 0
Number of Grain Boundaries?: 0
Number of Twin Boundaries?: 1
What percentage of samples should have a void at the grain boundary?: 0
How many samples of the same angle?: 1
How many samples (must be even number)?: 100
Creating sample pair 098 and 099
NP Diameter: 6.600000,  Number of electrons: 473
